# Collection setup and data load

## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")

print(WEAVIATE_URL)
print(WEAVIATE_KEY)

if(WEAVIATE_URL == "UPDATE_ME_WEAVIATE_URL"):
    raise Exception("Please update .env and Restart the notebook (see Restart button, next to Run All)")

## Connect to Weaviate

You need to pass in your Weaviate Cloud URL and KEY.

In [ ]:
import weaviate
from weaviate.classes.init import Auth
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),

    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=2, query=45, insert=120),  # Values in seconds
    # )
)

client.is_ready()

## A bit bigger example - 2k objects

### Load data

In [ ]:
import json

with open("./wiki-2k.json") as file:
    data_2k = json.load(file)

print(json.dumps(data_2k[0:2], indent=2))

### Create a collection with Named Vectors and SourceProperties

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

def create_wiki_collection():
    if client.collections.exists("Wiki"):
        client.collections.delete("Wiki")

    # Create a collection here - with Weaviate vectorizer and define source properties
    client.collections.create(
        name="Wiki",

        vectorizer_config=[
            # NOTE: we are using NamedVectors here
            Configure.NamedVectors.text2vec_weaviate(
                name="main_vector",

                model="Snowflake/snowflake-arctic-embed-l-v2.0",

                # TODO: set source properties to "title" and "text"
                # source_properties=[] # which properties should be used to generate a vector
            )
        ],

        # Example: how to define property schema (Optional)
        # properties=[  
        #     Property(name="title", data_type=DataType.TEXT),
        #     Property(name="text", data_type=DataType.TEXT),
        #     Property(name="url", data_type=DataType.TEXT),
        #     Property(name="wiki_id", data_type=DataType.TEXT),
        # ],
    )

create_wiki_collection()

### Import data - 2k objects with Batch

Batch speeds up the import process by grouping objects to be added in bigger batch groups.

Batch creates an internal buffer to collect objects to be added.<br>
Each time the buffer count reaches `batch_size`, batch sends the new objects to Weaviate.

Types of batch:
* `dynamic` - let batch calculate the optimal batch_size based on detected latency
* `fixed_size` - provide a fixed batch_size
* `rate_limit` - limit the number of requests (per minute), useful for working with models with a rate limit

### Take 1 – import sample 100

In [ ]:
from tqdm import tqdm

sample_100 = data_2k[0:100]

wiki = client.collections.get("Wiki")

# TODO: setup dynamic batch
# loop through the sample_100 data
# add each object to the batch

print(f"Wiki count: {len(wiki)}")

In [ ]:
# check for errors
if(len(wiki.batch.failed_objects)>0):
    print("Import complete with errors")
    for err in wiki.batch.failed_objects:
        print(err)
else:
    print("Import complete with no errors")

### Take 2 – import sample 100 – with UUID

To avoid inserting duplicates, you can generate a UUID based on the whole object or a unique property.

In [ ]:
from weaviate.util import generate_uuid5

print(generate_uuid5("This UUID is always the same"))
print(generate_uuid5("This UUID is always the same"))
print(generate_uuid5("This UUID is always the same"))
print("====================================")

print(generate_uuid5("This UUID is different"))
print(generate_uuid5("This UUID is different"))
print("====================================")

obj1 = { "title": "this is an object", "count": 1 }
obj2 = { "title": "this is an object", "count": 2 }
print(generate_uuid5(obj1))
print(generate_uuid5(obj2))


In [ ]:
# recreate the collection to start again
create_wiki_collection()

> Rerun the import script multiple times.

> Starting from the second run, the script should finish a lot faster, and the wiki count shouldn't increase.

In [ ]:
from tqdm import tqdm
from weaviate.util import generate_uuid5

sample_100 = data_2k[0:100]

wiki = client.collections.get("Wiki")

with wiki.batch.fixed_size(batch_size=50, concurrent_requests=2) as batch:
    for item in tqdm(sample_100):
        # TODO: generate an id from item["wiki_id"]
        # id = 

        batch.add_object(
            item,
            # TODO: provide the new id here 
            # uuid=
        )

print(f"Wiki count: {len(wiki)}")

### Take 2 - import the rest of the data - but break if multiple errors

In [ ]:
from tqdm import tqdm
from weaviate.util import generate_uuid5

wiki = client.collections.get("Wiki")

with wiki.batch.fixed_size(batch_size=600, concurrent_requests=2) as batch:
    for item in tqdm(data_2k):
        id = generate_uuid5(item["wiki_id"])
        batch.add_object(item, uuid=id)

        # Check number of errors while running
        if(batch.number_errors > 10):
            print("Errors during batch import")
            break

### Check for errors

In [ ]:
if(len(wiki.batch.failed_objects)>0):
    print("Import complete with errors")
    for err in wiki.batch.failed_objects:
        print(err)
else:
    print("Import complete with no errors")

## Bonus - iterate through all collection data

The client has a built-in function that allows you to iterate through all collection data.

In [ ]:
wiki = client.collections.get("Wiki")

counter = 100

for item in wiki.iterator():
    print(item.properties)

    if (counter == 0): break
    
    counter -= 1

You can also get `vector embeddings`, by using `include_vector`.

In [ ]:
counter = 10

for item in wiki.iterator(include_vector=True):
    print(item.properties)
    print(item.vector)

    if (counter == 0): break
    
    counter -= 1

## Close the client

In [ ]:
client.close()